In [1]:
import laspy, os
import numpy as np
from tqdm.auto import tqdm
from osgeo import gdal
from glob import glob

from PointUtils import getModelGrid_LAS

In [2]:
# PATH TO PYTORCH WEIGHTS
model_dir = r"data\012024.pth"

# PATH WITH LAZ FILES
input_dir = r"D:\Desktop\Roughness\SpringCreek\data\Terrain\\"

# PATH TO OUTPUT TIF FILES
output_dir = r"D:\Desktop\Roughness\SpringCreek\data\roughness\\"

# WILDCARD TO MATCH POINT CLOUD EXTENSIONS
laspy_wildcard = ".laz"

Processing 1


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/9255724 [00:00<?, ?it/s]

  0%|          | 0/2254329 [00:00<?, ?it/s]

  0%|          | 0/564 [00:00<?, ?it/s]

Assemble a list of files to process. Filter by which files already exist in output folder; if any tiles have already been processed, skip them.

In [ ]:
full_list = glob(input_dir+"*"+laspy_wildcard)
proc_list = []

for fn in full_list:
    raster_name = f"{output_dir}{os.path.basename(fn)[:-4]}.tif"
    if os.path.exists(raster_name):
        continue
    else:
        proc_list.append(fn)

Process LAZ tiles in directory

In [ ]:
print(f"Processing {len(proc_list)}")

for fn in tqdm(proc_list, total=len(proc_list)):
    raster_name = f"{output_dir}{os.path.basename(fn)[:-4]}.tif"
    
    model_grid, diagnostics = getModelGrid_LAS(fn, model_dir, 
                                           ground_class=2,
                                           save=raster_name,
                                           resolution=1,
                                           batch_size=4000)